In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
from keras.layers import Embedding,SpatialDropout1D,Input,Dropout,Activation,Bidirectional,GlobalMaxPooling1D,Dense,LSTM,Flatten
from keras.models import Sequential,Model
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import roc_auc_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['sampleSubmission.csv', 'unlabeledTrainData.tsv', 'testData.tsv', 'labeledTrainData.tsv']


In [2]:
df=pd.read_csv('../input/labeledTrainData.tsv',sep='\t',)

In [3]:
df.head()
df=df.drop(labels=['id'],axis=1)
df.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
df.sentiment.unique()

array([1, 0])

In [5]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
stpwords=set(stopwords.words('english'))
lematizer=WordNetLemmatizer()

In [8]:
def clean_text(text):
    text=re.sub(r'[^\w\s]','',text)
    text=text.lower().split()
    text=[lematizer.lemmatize(token) for token in text]
    text=[w for w in text if w not in stpwords]
    text=" ".join(text)
    return text



In [9]:
df['processed_reviews']=df.review.apply(lambda x:clean_text(x))

In [10]:
df.head()

,sentiment,review,processed_reviews
0,1,With all this stuff going down at the moment w...,stuff going moment mj ive started listening mu...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war world timothy hines entertaining f...
2,0,The film starts with a manager (Nicholas Bell)...,film start manager nicholas bell giving welcom...
3,0,It must be assumed that those who praised this...,must assumed praised film greatest filmed oper...
4,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious 80 ex...


In [13]:
df.review[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [14]:
df.processed_reviews[0]

'stuff going moment mj ive started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought wa really cool eighty maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema wa originally released ha subtle message mjs feeling towards press also obvious message drug bad mkaybr br visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fan would say made fan true really nice himbr br actual feature film bit finally start 20 minute excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord want mj dead bad beyond mj overheard plan nah joe pescis character ranted wanted people know supplying drug etc dunno maybe hate mjs musicbr br lot cool thing like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usua

In [15]:
from keras.preprocessing.text import Tokenizer

In [17]:
max_features=8000
tokenizer=Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df.processed_reviews)
list_tokenized_train=tokenizer.texts_to_sequences(df.processed_reviews)

In [18]:
list_tokenized_train[0]

[436,
 85,
 153,
 113,
 531,
 2420,
 136,
 71,
 908,
 482,
 197,
 197,
 186,
 61,
 13,
 659,
 1855,
 80,
 102,
 3,
 17,
 546,
 3555,
 186,
 15,
 231,
 594,
 2293,
 1158,
 59,
 4004,
 59,
 357,
 4,
 271,
 85,
 16,
 329,
 3,
 1625,
 530,
 7,
 1162,
 540,
 316,
 793,
 3137,
 22,
 467,
 540,
 763,
 24,
 1,
 1783,
 1027,
 173,
 371,
 1425,
 775,
 2329,
 6,
 487,
 85,
 585,
 60,
 262,
 109,
 414,
 160,
 2,
 118,
 14,
 44,
 30,
 118,
 194,
 17,
 230,
 1889,
 1,
 638,
 357,
 4,
 116,
 310,
 142,
 776,
 103,
 3361,
 1018,
 303,
 759,
 953,
 850,
 763,
 1342,
 61,
 266,
 24,
 591,
 820,
 759,
 10,
 365,
 21,
 35,
 763,
 477,
 186,
 585,
 7073,
 1,
 66,
 546,
 29,
 6,
 1429,
 335,
 1520,
 133,
 1835,
 1444,
 303,
 22,
 67,
 131,
 4664,
 3782,
 272,
 1267,
 24,
 303,
 510,
 67,
 585,
 658,
 5,
 135,
 159,
 495,
 133,
 630,
 3179,
 1165,
 637,
 3845,
 1,
 1198,
 121,
 2,
 21,
 6,
 5,
 443,
 76,
 32,
 21,
 524,
 155,
 51,
 128,
 62,
 7074,
 540,
 3334,
 1405,
 2,
 88,
 371,
 1425,
 258,
 5,
 893,
 2

In [23]:
sorted(tokenizer.word_counts.items(),key=lambda x:x[1],reverse=True)

[('br', 57145),
 ('movie', 49596),
 ('wa', 48014),
 ('film', 45186),
 ('one', 26667),
 ('like', 20105),
 ('ha', 16849),
 ('time', 14869),
 ('good', 14609),
 ('character', 13807),
 ('story', 12649),
 ('even', 12504),
 ('get', 12424),
 ('would', 12135),
 ('make', 11915),
 ('see', 11762),
 ('really', 11664),
 ('scene', 10232),
 ('much', 9585),
 ('well', 9489),
 ('people', 9219),
 ('also', 9073),
 ('great', 8925),
 ('bad', 8913),
 ('first', 8858),
 ('dont', 8494),
 ('way', 8457),
 ('show', 8358),
 ('thing', 8024),
 ('made', 7990),
 ('could', 7713),
 ('think', 7660),
 ('go', 7404),
 ('life', 7386),
 ('know', 6930),
 ('watch', 6896),
 ('two', 6649),
 ('many', 6640),
 ('love', 6635),
 ('actor', 6566),
 ('seen', 6529),
 ('plot', 6529),
 ('look', 6451),
 ('say', 6433),
 ('never', 6425),
 ('little', 6388),
 ('end', 6313),
 ('acting', 6296),
 ('year', 6271),
 ('best', 6269),
 ('doe', 5901),
 ('ever', 5804),
 ('take', 5614),
 ('come', 5602),
 ('still', 5592),
 ('man', 5577),
 ('better', 5548),
 ('

In [24]:
tokenizer.word_index

{'br': 1,
 'movie': 2,
 'wa': 3,
 'film': 4,
 'one': 5,
 'like': 6,
 'ha': 7,
 'time': 8,
 'good': 9,
 'character': 10,
 'story': 11,
 'even': 12,
 'get': 13,
 'would': 14,
 'make': 15,
 'see': 16,
 'really': 17,
 'scene': 18,
 'much': 19,
 'well': 20,
 'people': 21,
 'also': 22,
 'great': 23,
 'bad': 24,
 'first': 25,
 'dont': 26,
 'way': 27,
 'show': 28,
 'thing': 29,
 'made': 30,
 'could': 31,
 'think': 32,
 'go': 33,
 'life': 34,
 'know': 35,
 'watch': 36,
 'two': 37,
 'many': 38,
 'love': 39,
 'actor': 40,
 'seen': 41,
 'plot': 42,
 'look': 43,
 'say': 44,
 'never': 45,
 'little': 46,
 'end': 47,
 'acting': 48,
 'year': 49,
 'best': 50,
 'doe': 51,
 'ever': 52,
 'take': 53,
 'come': 54,
 'still': 55,
 'man': 56,
 'better': 57,
 'work': 58,
 'part': 59,
 'find': 60,
 'want': 61,
 'give': 62,
 'something': 63,
 'back': 64,
 'im': 65,
 'lot': 66,
 'director': 67,
 'performance': 68,
 'doesnt': 69,
 'real': 70,
 'watching': 71,
 'woman': 72,
 'though': 73,
 'didnt': 74,
 'play': 75,
 

In [25]:
index2word={j:i for i,j in tokenizer.word_index.items()}

In [26]:
" ".join([index2word.get(w) for w in list_tokenized_train[0]])

'stuff going moment ive started listening music watching odd documentary watched watched maybe want get certain insight guy thought wa really cool eighty maybe make mind whether guilty innocent part biography part feature film remember going see cinema wa originally released ha subtle message feeling towards press also obvious message drug bad br visually impressive course michael jackson unless remotely like anyway going hate find boring may call making movie fan would say made fan true really nice himbr br actual feature film bit finally start 20 minute smooth criminal sequence joe convincing powerful drug lord want dead bad beyond plan joe character wanted people know drug etc maybe hate musicbr br lot cool thing like turning car robot whole speed demon sequence also director must patience saint came filming bad sequence usually director hate working one kid let alone whole bunch performing complex dance scenebr br bottom line movie people like one level another think people stay aw

In [27]:
maxlen=150
X_t=pad_sequences(list_tokenized_train,maxlen=maxlen)

In [28]:
y=df.sentiment

In [29]:
X_t[5]  #so padding is "pre" and truncating is also 'pre' i.e is in the starting

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   26,   35,   21,   32,
         24,    2,  100,   96,    9,   42,    9,  112,  374,  874, 1159,
         51, 1061,  257,  157, 2206, 1948,    2,    6, 3914,    9,  607,
       1467, 1159,  319, 2924,    2,  298,  390,  587,  319, 1467, 1159,
         16,    5,   66,   57,  266, 2843, 3219], d

In [30]:
embed_size=128
model=Sequential()
model.add(Embedding(max_features,embed_size))
model.add(Bidirectional(LSTM(32,return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dense(20,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

batchsize=100
epochs=3
model.fit(X_t,y,batch_size=batchsize,epochs=epochs,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 99s 5ms/step - loss: 0.4587 - acc: 0.7866 - val_loss: 0.3093 - val_acc: 0.8680
Epoch 2/3
20000/20000 [==============================] - 95s 5ms/step - loss: 0.2316 - acc: 0.9154 - val_loss: 0.3008 - val_acc: 0.8762
Epoch 3/3
20000/20000 [==============================] - 96s 5ms/step - loss: 0.1581 - acc: 0.9468 - val_loss: 0.3278 - val_acc: 0.8704


In [31]:
test_df=pd.read_csv('../input/testData.tsv',sep='\t')

In [33]:
test_df.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [34]:
test_df['processed_review']=test_df.review.apply(lambda x:clean_text(x))

In [35]:
test_df.head()

,id,review,processed_review
0,12311_10,Naturally in a film who's main themes are of m...,naturally film main theme mortality nostalgia ...
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kid saw tonight child loved one point ki...
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark left impression several different ...
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life filmed ...


In [36]:
list_tokenized_test=tokenizer.texts_to_sequences(test_df.processed_review)

In [40]:
len(list_tokenized_test[0])

65

In [41]:
X_test=pad_sequences(list_tokenized_test,maxlen=maxlen)

In [43]:
X_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0, 1769,    4,  189,
        409, 4262, 1616, 2860,  275, 1585, 1275,  434,  779,  225,  993,
          5,  104,    4,  168,  255,  922, 5068, 1608,   10,  277, 1582,
        378, 2515, 1095,  650,  158, 1203,   28,  675, 1291,   28,  792,
       5190,   28,  475,   28, 2211, 1769,  234,   11, 6361,    4, 1460,
         30, 2128,  307, 4475, 2884,  298,  374, 5466,   15, 6247, 4139,
        723, 5511,  258,  771, 3077, 3719,  758], d

In [44]:
test_predict=model.predict(X_test)

In [45]:
test_predict

array([[0.99430346],
       [0.0038621 ],
       [0.02037188],
       ...,
       [0.8820067 ],
       [0.9892207 ],
       [0.42154473]], dtype=float32)

In [49]:
y_pred=(test_predict>0.5)*1

In [54]:
y_pred=np.squeeze(y_pred)

In [56]:
y_pred.shape

(25000,)

In [57]:
output=pd.DataFrame({'id':test_df['id'],'sentiment':y_pred})

In [58]:
output

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,1
5,2913_8,0
6,4396_1,0
7,395_2,1
8,10616_1,0
9,9074_9,0


In [ ]:
output.to_csv("submission.csv", index = False, quoting = 3)